In [1]:
import pandas as pd

df = pd.read_excel("./base_de_donnees.xlsx") 
texts = df['Contenu'].tolist()
contexts = df['Contexte'].map(lambda x: x.replace('.x.', '.')).tolist()
print(df.shape)
print('*' * 120)
for source in df.Source.unique():
    print(f"Source {source}: {len(df[df['Source']==source])}")

print('*' * 120)
print(contexts[0])

(4132, 3)
************************************************************************************************************************
Source Etablissement recevant des travailleurs: 299
Source Réglementation sécurité et incendie. Etablissement recevant du public: 44
Source Règlement de sécurité contre les risques d'incendie et de panique. Etablissements recevant du public: 1161
Source Code de la construction et de l'habitation: 2628
************************************************************************************************************************
Obligations du maître d'ouvrage pour la conception des lieux de travail  (Articles R4211-1 à R4217-2). Principes généraux (Articles R4211-1 à R4211-5). Champ d'application et définitions (Articles R4211-1 à R4211-2). Article R4211-1. Créé par Décret n°2008-244 du 7 mars 2008 - art. (V). 


In [2]:
from sentence_transformers import SentenceTransformer
import torch

# Charger SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# Calcul des embeddings
df["Embedding"] = df['Contenu'].apply(lambda x: model.encode(x))
text_embeddings = torch.stack([torch.tensor(embedding) for embedding in df["Embedding"]])

C:\Users\Nedved\anaconda3\envs\chromadb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import numpy as np


user_query = "Je dois installer des dispositifs de désenfumage. A quoi dois je faire attention"
user_query = "Je dois faire des modification dans un établissement recevant le public. Par qui l'autorisation est elle délivrée"

query_embeddings = torch.tensor(model.encode(user_query))

similarities = torch.nn.functional.cosine_similarity(query_embeddings, text_embeddings, dim=1)

most_similar_index = torch.argmax(similarities).item()

top_k = 3
top_k_indices = torch.argsort(similarities, descending=True)[:top_k]
print(f"User query: {user_query}\n\n")

for idx in top_k_indices:
    idx = idx.item() 
    text = df.loc[idx, 'Contenu']
    context = df.loc[idx, 'Contexte']
    source = df.loc[idx, 'Source']
    similarity_coeff = similarities[idx]
    print('-' * 120)
    print(f"Cosine Similarity: {similarity_coeff:.4f}\n\nSource plus proche:{df.loc[idx, 'Source']}\n\n Contexte: {df.loc[idx, 'Contexte']}")
    print(f"\nTexte: {df.loc[idx, 'Contenu']}")

User query: Je dois faire des modification dans un établissement recevant le public. Par qui l'autorisation est elle délivrée


------------------------------------------------------------------------------------------------------------------------
Cosine Similarity: 0.6960

Source plus proche:Code de la construction et de l'habitation

 Contexte: Article R122-8.x. Construction, entretien et rénovation des bâtiments.x. ENCADREMENT DE LA CONCEPTION, DE LA RÉALISATION ET DE L'EXPLOITATION DES BÂTIMENTS.x. Procédures administratives.x. Déclarations et autorisations.x. Autorisations applicables aux établissements recevant du public.x. Compétence.x. Création Décret n°2021-872 du 30 juin 2021 - art..x. 

Texte: L'autorisation ne peut être délivrée que si les travaux projetés sont conformes : a)  Aux règles d'accessibilité aux personnes handicapées prescrites, pour la construction ou la création d'un établissement recevant du public, à la section 3 du chapitre II du titre VI ou, pour l'aménag

In [4]:
import chromadb
#1. Install

# !pip install chromadb

#2. Create a Chroma Client
chroma_client = chromadb.PersistentClient(path="./data/chroma_db3")

#3. Create a collection
collection = chroma_client.get_or_create_collection(name="articles_reglementaires")

In [5]:
chroma_client.list_collections()

['articles_reglementaires']

In [6]:
for index, row in df.iterrows():
    unique_id = f"chunk_{index}"

    metadata = {
        "Contexte": row.get("Contexte", ""),
        "Source": row.get("Source", ""),
        "Contenu": row.get("Contenu", ""),
    }

    collection.add(
        documents=[row["Contenu"]],
        embeddings=[row["Embedding"]],
        metadatas=[metadata],
        ids=[unique_id]  
    )

Add of existing embedding ID: chunk_0
Add of existing embedding ID: chunk_1
Add of existing embedding ID: chunk_2
Add of existing embedding ID: chunk_3
Add of existing embedding ID: chunk_4
Add of existing embedding ID: chunk_5
Add of existing embedding ID: chunk_6
Add of existing embedding ID: chunk_7
Add of existing embedding ID: chunk_8
Add of existing embedding ID: chunk_9
Add of existing embedding ID: chunk_10
Add of existing embedding ID: chunk_11
Add of existing embedding ID: chunk_12
Add of existing embedding ID: chunk_13
Add of existing embedding ID: chunk_14
Add of existing embedding ID: chunk_15
Add of existing embedding ID: chunk_16
Add of existing embedding ID: chunk_17
Add of existing embedding ID: chunk_18
Add of existing embedding ID: chunk_19
Add of existing embedding ID: chunk_20
Add of existing embedding ID: chunk_21
Add of existing embedding ID: chunk_22
Add of existing embedding ID: chunk_23
Add of existing embedding ID: chunk_24
Add of existing embedding ID: chunk

In [7]:
# Vérification du nombre de documents
print(f"La collection '{collection.name}' a été créée avec succès.")
print('Il y a', collection.count(), 'documents dans la collection.')

La collection 'articles_reglementaires' a été créée avec succès.
Il y a 4132 documents dans la collection.


In [8]:
#5. Query the collection
query = "A quel type de bâtiment s'applique le code du travail ? "
results = collection.query(query_texts=[query], n_results=5)

for i, result in enumerate(results['documents'][0]):
    print(f"Résultat {i+1}:")
    print(f"Texte : {result}")
    print(f"Métadonnées : {results['metadatas'][0][i]}")
    print()


Résultat 1:
Texte : Pour l'application des articlesL. 141-1 à L. 141-4, les dispositions réglementaires applicables aux bâtiments ou parties de bâtiment à usage professionnel figurent auchapitre VI du titre I du livre II de la quatrième partie du code du travail.
Métadonnées : {'Contenu': "Pour l'application des articlesL. 141-1 à L. 141-4, les dispositions réglementaires applicables aux bâtiments ou parties de bâtiment à usage professionnel figurent auchapitre VI du titre I du livre II de la quatrième partie du code du travail.", 'Contexte': "Article R144-1.x. Construction, entretien et rénovation des bâtiments.x. SÉCURITÉ DES PERSONNES CONTRE LES RISQUES D'INCENDIE.x. BÂTIMENTS À USAGE PROFESSIONNEL.x. Agrément des contrôleurs techniques.x. Diagnostic portant sur les déchets issus de rénovations et de démolitions.x. Création Décret n°2021-872 du 30 juin 2021 - art..x. ", 'Source': "Code de la construction et de l'habitation"}

Résultat 2:
Texte : Pour l'application de la garantie pré

In [ ]:
model = model.to('cpu')

In [ ]:
# la transformation en embedding aurait pu se faire directement avec Chroma

# enregistrement de la clé API
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("Clé API récupérée avec succès !")
else:
    print("Erreur : La clé API n'a pas été trouvée.")

from openai import OpenAI
openai.api_key = api_key

# Configuration de Chroma comme base vectorielle
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma(
    collection_name="articles_reglementaires",
    persist_directory="./data/chroma_db3",  # Chemin où nous avons stocké nos données
    embedding_function=embedding_model
)

# Affichage des résultats

for doc in results:
    print("Document :", doc.page_content)
    print("Métadonnées :", doc.metadata)


# Création du retriever

retriever = vectorstore.as_retriever(
    search_type="similarity",  # Type de recherche
    search_kwargs={"k": 5}  # Nombre de résultats à retourner
)

# Nous ré-essayons la query

query = "A quel type de bâtiment s'applique le code du travail ?"
results = retriever.invoke(query)
print(results)

# Définition du llm utilisé

llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

# Définition de la mémoire

memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True, output_key='answer')

# Définition de la chaine de RAG

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    retriever=retriever,
    return_source_documents=True
)

# Essai de la chaine de RAG

rag_chain("À quel type de bâtiment s'applique le code du travail ?")
